In [63]:
# from langchain.llms import google_palm\
  
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os

load_dotenv()

llm = ChatGroq(
  model = "llama-3.1-70b-versatile",
  temperature = 0,
  groq_api_key = os.getenv("GROQ_API_KEY"),
)

# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

print(llm.invoke("Write a poem on Love of Briyani ").content)



Aromatic scents that fill the air,
A dish so fine, beyond compare,
Briyani, oh how you delight,
A flavorful treat, both day and night.

Your basmati rice, a treasure rare,
Mixed with spices, a love to share,
The tender meat, or veggies so fine,
A symphony of taste, all divine.

The onions, ginger, and garlic too,
Dance on the tongue, a flavorful brew,
The cardamom, cinnamon, and cloves so sweet,
A love letter to the senses, can't be beat.

In every bite, a story's told,
Of love and care, of traditions old,
A dish that brings, people near,
A symbol of love, that casts out fear.

Oh, Briyani, how you bring us cheer,
A comfort food, that banishes all fear,
You're more than just, a meal to eat,
A love that's shared, a bond to greet.

So here's to you, dear Briyani mine,
A love that's pure, a taste divine,
May your flavors, forever reign,
And in our hearts, your love remain.


In [64]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "2004"
db_host = "localhost"
db_name = "martians_tshirt"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [65]:
# from langchain_experimental.sql import SQLDatabaseChain
# from decimal import Decimal

# db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True,return_direct = True)

# response = db_chain.invoke("What is the total revenue from Small sized t-shirts?")

# sql_result = response['result']



## CUSTOM SQL DB CHAIN

In [66]:
from langchain_experimental.sql import SQLDatabaseChain
import re
from decimal import Decimal

class CustomSQLDatabaseChain(SQLDatabaseChain):
    def invoke(self, query: str, return_only_outputs: bool = True):
        # Run the original chain to get the SQL query result
        response = super().invoke(query)
        print(response)
        # Extract and process the result
        sql_result = response['result']
        
        # Apply regex to extract the Decimal value and convert it to float
        match = re.search(r"Decimal\('([\d\.]+)'\)", sql_result)
        if match:
            result = float(Decimal(match.group(1)))
            response['result'] = result
        else:
            response['result'] = "Unexpected result format: " + sql_result
        
        # If return_only_outputs is True, only return the result value
        if return_only_outputs:
            return response['result']
        
        # Otherwise, return the full response
        return response


In [67]:
# Initialize the custom database chain
db_chain = CustomSQLDatabaseChain.from_llm(llm, db, verbose=True, return_direct=True)

# Get the query result and processed answer
response = db_chain.invoke("What is the total revenue from Small sized t-shirts?")





> Entering new CustomSQLDatabaseChain chain...
What is the total revenue from Small sized t-shirts?
SQLQuery:

Question: What is the total revenue from Small sized t-shirts?
SQLQuery: 
SELECT SUM(`price` * `stock_quantity`) AS total_revenue 
FROM `t_shirts` 
WHERE `size` = 'S';
SQLResult: [(Decimal('25345'),)]
> Finished chain.
{'query': 'What is the total revenue from Small sized t-shirts?', 'result': "[(Decimal('25345'),)]"}


In [68]:
print(response)  # Should print: 822


25345.0


In [69]:
qns1 = db_chain.invoke("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'L'")
qns1



> Entering new CustomSQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'L'
SQLQuery:

Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'L'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'Red' AND `size` = 'L'
SQLResult: [(Decimal('55'),)]
> Finished chain.
{'query': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'L'", 'result': "[(Decimal('55'),)]"}


55.0

In [70]:
qns2 = db_chain.invoke("How much is the total price of the inventory for all s size t-shirts after applying discounts?")
qns2 



> Entering new CustomSQLDatabaseChain chain...
How much is the total price of the inventory for all s size t-shirts after applying discounts?
SQLQuery:Question: How much is the total price of the inventory for all s size t-shirts after applying discounts?
SQLQuery: 
SELECT 
    SUM(`price` * (1 - `pct_discount` / 100) * `stock_quantity`) AS total_price
FROM 
    t_shirts
JOIN 
    discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id
WHERE 
    `size` = 'S'
SQLResult: [(Decimal('3680.600000'),)]
> Finished chain.
{'query': 'How much is the total price of the inventory for all s size t-shirts after applying discounts?', 'result': "[(Decimal('3680.600000'),)]"}


3680.6

In [71]:
qns3 = db_chain.invoke("""SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id (use exact query)""")
qns3



> Entering new CustomSQLDatabaseChain chain...
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id (use exact query)
SQLQuery:Question: SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id

SQLQuery: 
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue 
FROM 
(select sum(`price`*`stock_quantity`) as total_amount, `t_shirt_id` 
from t_shirts 
where `brand` = 'Levi' 
group by `t_shirt_id`) a 
LEFT JOIN discounts 
ON a.`t_shirt_id` = discounts.`t_shirt_id`
SQLResult: [(Decimal('11136.100000'),)]
> Finished

11136.1

In [72]:
qns4 = db_chain.invoke("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")
qns4



> Entering new CustomSQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) AS total_value FROM t_shirts WHERE `brand` = 'Levi' LIMIT 5;
SQLResult: [(Decimal('12913'),)]
> Finished chain.
{'query': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'", 'result': "[(Decimal('12913'),)]"}


12913.0

In [73]:
qns5 = db_chain.invoke("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")
qns5



> Entering new CustomSQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 1;
SQLResult: [(Decimal('217'),)]
> Finished chain.
{'query': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'", 'result': "[(Decimal('217'),)]"}


217.0

In [74]:
qns6 = db_chain.invoke("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'")
qns6



> Entering new CustomSQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'
SQLQuery:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' LIMIT 5;
SQLResult: [(Decimal('681'),)]
> Finished chain.
{'query': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'", 'result': "[(Decimal('681'),)]"}


681.0

In [75]:
print(qns1,qns2,qns3,qns4,qns5,qns6)

55.0 3680.6 11136.1 12913.0 217.0 681.0


In [76]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "55.0"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "3680.6"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "11136.1"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "12913.0"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "217.0"
     },
    {'Question': "How many Nike's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "681.0"
     }
]

In [77]:
# few_shots = [
#     {
#         'Question': "How many t-shirts do we have left for Nike in XS size and white color?",
#         'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
#         'SQLResult': str(qns1)
#     },
#     {
#         'Question': "How much is the total price of the inventory for all S-size t-shirts?",
#         'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
#         'SQLResult': str(qns2)
#     },
#     {
#         'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
#         'SQLQuery': """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
#         (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
#         group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id""",
#         'SQLResult': str(qns3)
#     },
#     {
#         'Question': "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
#         'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
#         'SQLResult': str(qns4)
#     },
#     {
#         'Question': "How many white color Levi's shirt I have?",
#         'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
#         'SQLResult': str(qns5)
#     },
#     {
#         'Question': "How many Nike's shirt I have?",
#         'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'",
#         'SQLResult': str(qns6)
#     }
# ]


In [78]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

e = embeddings.embed_query("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")
len(e)

1536

In [79]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 55.0",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 3680.6",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 11136.1",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Lev

### CHROMA

In [80]:
from langchain.vectorstores import Chroma

vectorStore = Chroma.from_texts(to_vectorize,embedding=embeddings,metadatas=few_shots)

In [81]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
  vectorstore=vectorStore,
  k=2
)

example_selector.select_examples({"Question":"how many Adidas t-shirts i have ;eft in my store ?"})

[{'Answer': '681.0',
  'Question': "How many Nike's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '681.0',
  'Question': "How many Nike's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike'",
  'SQLResult': 'Result of the SQL query'}]

In [82]:
from langchain.prompts import PromptTemplate


example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}",
)

-----------

----------

In [83]:
from langchain.prompts import FewShotPromptTemplate


In [84]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [85]:
table_data = db.table_info

print(table_data)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [86]:
from langchain.prompts import FewShotPromptTemplate

FewShot_Prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""
    

You are a MySQL expert. Given an input question, your job is to:
1. Create a syntactically correct SQL query.
2. Run the query and provide only the result of the SQL query as the output.
3. Only use the following tables:(-- discounts table
CREATE TABLE discounts (
    discount_id INTEGER NOT NULL AUTO_INCREMENT, 
    t_shirt_id INTEGER NOT NULL, 
    pct_discount DECIMAL(5, 2), 
    PRIMARY KEY (discount_id), 
    CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
    CONSTRAINT discounts_chk_1 CHECK (pct_discount BETWEEN 0 AND 100)
);

-- t_shirts table
CREATE TABLE t_shirts (
    t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
    brand ENUM('Van Huesen', 'Levi', 'Nike', 'Adidas') NOT NULL, 
    color ENUM('Red', 'Blue', 'Black', 'White') NOT NULL, 
    size ENUM('XS', 'S', 'M', 'L', 'XL') NOT NULL, 
    price INTEGER, 
    stock_quantity INTEGER NOT NULL, 
    PRIMARY KEY (t_shirt_id), 
    CONSTRAINT t_shirts_chk_1 CHECK (price BETWEEN 10 AND 50)
);
)

The response format must be strictly:
Question: Question text
SQLQuery: SQL query
SQLResult: Result of the SQL query (Strictly the numerical value of SQLResult not the word SQLResult)
""",
    suffix="""
Only include the SQLResult in the response. Do not provide an Answer key or additional explanations.
Question: {input}
SQLQuery:
""",
    input_variables=["input", "table_info", "top_k"],
)


In [87]:
from langchain_experimental.sql import SQLDatabaseChain
from decimal import Decimal
import re

class NewCustomSql(SQLDatabaseChain):
    def invoke(self, query: str,return_only_outputs= False):
        # Run the original chain to get the SQL query result
        response = super().invoke(query)
        print(response)
        # Extract and process the result
        sql_result = response['result']
        
        # Apply regex to extract the Decimal value and convert it to float
        match = re.search(r"Decimal\('([\d\.]+)'\)|[\d\.]+", sql_result)
        if match:
            result = float(Decimal(match.group(1)))
            response['result'] = result
        else:
            response['result'] = "Unexpected result format: " + sql_result
        
        # If return_only_outputs is True, only return the result value
        if return_only_outputs:
            return response['result']
        
        # Otherwise, return the full response
        return response
    


In [88]:
new_chain = SQLDatabaseChain.from_llm(llm = llm,db = db,verbose = True,prompt = FewShot_Prompt)

new_chain.invoke("How many tshirts are left in total number of stock ?")





> Entering new SQLDatabaseChain chain...
How many tshirts are left in total number of stock ?
SQLQuery:

SQLQuery: SELECT sum(stock_quantity) FROM t_shirts
SQLResult: [(Decimal('3135'),)]
Answer:3135
> Finished chain.


{'query': 'How many tshirts are left in total number of stock ?',
 'result': '3135'}

In [89]:
db_chain.invoke("How many total Levi's T Shirt I have ?")




> Entering new CustomSQLDatabaseChain chain...
How many total Levi's T Shirt I have ?
SQLQuery:Question: How many total Levi's T Shirt I have ?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLResult: [(Decimal('609'),)]
> Finished chain.
{'query': "How many total Levi's T Shirt I have ?", 'result': "[(Decimal('609'),)]"}


609.0